# 🚀 Projeto Final 2: Sistema Inteligente de Análise de Documentos com Multi-Agentes

**Módulo 13 - LangChain v0.3**

*Pedro Guth - Expert em IA & AWS*

---

Tá, chegamos no segundo projeto final! Se no primeiro projeto criamos um assistente simples, agora vamos partir pro **nível hardcore**: um sistema multi-agente que analisa documentos, extrai insights e ainda gera relatórios automaticamente!

É tipo ter uma equipe de consultores trabalhando pros seus documentos 24/7! 📊

## 🎯 O que vamos construir?

Imagina que você tem uma pilha de documentos (PDFs, textos, relatórios) e precisa:

1. **Extrair informações-chave** automaticamente
2. **Analisar sentimentos** e tendências 
3. **Gerar resumos executivos**
4. **Criar visualizações** dos dados encontrados
5. **Responder perguntas** sobre qualquer documento

É como ter um **ChatGPT turbinado** que não só conversa, mas também vira analista de dados!

**Dica!** Este projeto combina TUDO que vimos: RAG, Agents, Memory, Chains e muito mais!

## 🏗️ Arquitetura do Sistema

Nosso sistema terá **4 agentes especialistas**:

```mermaid
graph TD
    A[Documento] --> B[Agente Extrator]
    B --> C[Agente Analisador]
    B --> D[Agente Visualizador]
    B --> E[Agente Perguntador]
    C --> F[Relatório Final]
    D --> F
    E --> F
```

Cada agente tem uma **especialidade**, tipo uma empresa de consultoria onde cada um faz sua parte!

In [ ]:
# Primeiro, vamos instalar e importar tudo que precisamos
!pip install langchain langchain-google-genai chromadb pypdf python-docx matplotlib seaborn wordcloud textstat plotly -q

print("🎉 Instalação concluída! Bora pro código!")

In [ ]:
# Imports essenciais
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import json

# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import BaseOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory

print("📚 Todas as bibliotecas importadas com sucesso!")

## 🔑 Configuração Inicial

Antes de começar, precisamos configurar nossa chave da API do Google. É como dar a chave do carro pro nosso sistema dirigir sozinho!

In [ ]:
# Configuração da API Key
from google.colab import userdata

# Se estiver no Colab, use a linha abaixo:
api_key = userdata.get('GOOGLE_API_KEY')

# Se estiver rodando local, descomente e coloque sua chave:
# api_key = "sua_chave_aqui"

os.environ["GOOGLE_API_KEY"] = api_key

# Inicializando nosso modelo principal
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.3,  # Meio termo entre criatividade e precisão
    google_api_key=api_key
)

print("🤖 LLM configurado! Gemini 2.0 Flash pronto pra trabalhar!")

## 📄 Carregamento e Processamento de Documentos

Vamos criar um sistema que consegue ler diferentes tipos de documento. É tipo ensinar nosso sistema a ler português, inglês, PDF, Word... tudo!

In [ ]:
class DocumentProcessor:
    def __init__(self):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )
        self.documents = []
        self.chunks = []
    
    def load_text_file(self, file_path):
        """Carrega arquivo de texto simples"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            return content
        except:
            print(f"❌ Erro ao carregar {file_path}")
            return None
    
    def process_text(self, text, source="texto"):
        """Processa texto em chunks menores"""
        if text:
            # Cria chunks do texto
            chunks = self.text_splitter.split_text(text)
            
            # Adiciona metadados a cada chunk
            processed_chunks = []
            for i, chunk in enumerate(chunks):
                processed_chunks.append({
                    'content': chunk,
                    'source': source,
                    'chunk_id': i,
                    'word_count': len(chunk.split())
                })
            
            self.chunks.extend(processed_chunks)
            return processed_chunks
        return []

# Vamos criar um exemplo de documento para testar
sample_text = """
Relatório de Vendas - Q4 2024

Este relatório apresenta uma análise detalhada do desempenho de vendas do quarto trimestre de 2024.

Resumo Executivo:
As vendas do Q4 2024 apresentaram um crescimento significativo de 25% em relação ao mesmo período do ano anterior. 
O setor de tecnologia liderou o crescimento, representando 40% do faturamento total.

Principais Métricas:
- Receita total: R$ 2.5 milhões
- Crescimento: +25% YoY
- Novos clientes: 150
- Taxa de retenção: 85%

Desafios Identificados:
1. Concorrência acirrada no mercado de software
2. Necessidade de melhorar o atendimento ao cliente
3. Expansão para novos mercados geográficos

Oportunidades:
1. Lançamento de novos produtos em 2025
2. Parcerias estratégicas com empresas de tecnologia
3. Investimento em marketing digital

Conclusão:
O Q4 2024 foi um período de forte crescimento, mas é necessário focar na retenção de clientes e expansão de mercado para 2025.
"""

# Processando nosso documento de exemplo
processor = DocumentProcessor()
chunks = processor.process_text(sample_text, "relatorio_vendas_q4")

print(f"📊 Documento processado com sucesso!")
print(f"📝 Total de chunks: {len(chunks)}")
print(f"📄 Primeiro chunk: {chunks[0]['content'][:100]}...")

## 🧠 Sistema de Embeddings e Vector Store

Agora vamos transformar nossos textos em **vetores matemáticos**. É como dar uma "impressão digital" pra cada pedaço de texto, permitindo que o sistema encontre informações relacionadas rapidinho!

**Dica!** Lembra do Módulo 9? Estamos usando exatamente os conceitos de Vector Store que aprendemos lá!

In [ ]:
# Configurando embeddings e vector store
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=api_key
)

# Preparando textos e metadados para o Chroma
texts = [chunk['content'] for chunk in chunks]
metadatas = [{'source': chunk['source'], 'chunk_id': chunk['chunk_id']} for chunk in chunks]

# Criando o vector store
vectorstore = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    collection_name="documentos_projeto"
)

print("🔍 Vector Store criado com sucesso!")
print(f"📊 {len(texts)} documentos indexados")

# Testando uma busca
query = "receita e crescimento"
resultados = vectorstore.similarity_search(query, k=2)

print(f"\n🔎 Teste de busca para '{query}':")
for i, doc in enumerate(resultados):
    print(f"Resultado {i+1}: {doc.page_content[:100]}...")

## 🤖 Agente 1: Extrator de Informações

Nosso primeiro agente é especialista em **garimpar** informações importantes dos documentos. É tipo aquele amigo que sempre acha o que você precisa em qualquer bagunça!

Ele vai procurar:
- **Métricas numéricas** (vendas, percentuais, etc.)
- **Datas importantes**
- **Nomes de pessoas/empresas**
- **Palavras-chave relevantes**

In [ ]:
# Template para extração de informações
extraction_template = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um especialista em extração de informações de documentos.
    Sua missão é encontrar e organizar as informações mais importantes.
    
    Extraia as seguintes informações do texto:
    1. MÉTRICAS NUMÉRICAS (valores, percentuais, quantidades)
    2. DATAS E PERÍODOS importantes
    3. NOMES (pessoas, empresas, produtos)
    4. PALAVRAS-CHAVE relevantes
    5. TÓPICOS PRINCIPAIS
    
    Organize tudo em formato JSON claro e estruturado.
    """),
    ("human", "Texto para análise: {texto}")
])

# Criando nosso agente extrator
def extrair_informacoes(texto):
    """
    Extrai informações estruturadas do texto
    """
    chain = extraction_template | llm
    
    try:
        resposta = chain.invoke({"texto": texto})
        return resposta.content
    except Exception as e:
        return f"Erro na extração: {e}"

# Testando com nosso documento
texto_completo = " ".join([chunk['content'] for chunk in chunks])
info_extraida = extrair_informacoes(texto_completo)

print("📊 INFORMAÇÕES EXTRAÍDAS:")
print("=" * 50)
print(info_extraida)

## 📈 Agente 2: Analisador de Sentimentos e Tendências

Este agente é nosso **psicólogo de textos**! Ele analisa:
- **Sentimento geral** (positivo, negativo, neutro)
- **Nível de confiança** das informações
- **Tendências** mencionadas
- **Pontos de atenção** e oportunidades

**Dica!** É como ter um consultor que não só lê os números, mas entende as entrelinhas!

In [ ]:
# Template para análise de sentimentos
sentiment_template = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um analista expert em sentiment analysis e análise de tendências.
    
    Analise o texto e forneça:
    
    1. SENTIMENT SCORE: (0-100, onde 0=muito negativo, 50=neutro, 100=muito positivo)
    2. CONFIANÇA: Nível de certeza das informações (0-100)
    3. TENDÊNCIAS IDENTIFICADAS: Crescimento, declínio, estabilidade
    4. PALAVRAS-CHAVE EMOCIONAIS: Termos que indicam sentiment
    5. INSIGHTS: 3 principais insights sobre o contexto
    6. ALERTAS: Possíveis pontos de atenção
    
    Seja objetivo e use dados do texto para justificar sua análise.
    """),
    ("human", "Texto para análise: {texto}")
])

def analisar_sentimento(texto):
    """
    Analisa sentimento e tendências do texto
    """
    chain = sentiment_template | llm
    
    try:
        resposta = chain.invoke({"texto": texto})
        return resposta.content
    except Exception as e:
        return f"Erro na análise: {e}"

# Analisando nosso documento
analise_sentimento = analisar_sentimento(texto_completo)

print("🎭 ANÁLISE DE SENTIMENTO:")
print("=" * 50)
print(analise_sentimento)

## 📊 Agente 3: Gerador de Visualizações

Este é nosso **designer de dados**! Ele pega as informações extraídas e cria gráficos lindos e informativos.

Vamos extrair dados numéricos e criar visualizações automáticas!

In [ ]:
# Vamos extrair dados numéricos do nosso texto para visualizar
import re

def extrair_dados_numericos(texto):
    """
    Extrai números, percentuais e valores do texto
    """
    # Padrões para encontrar dados numéricos
    valores_reais = re.findall(r'R\$\s*([\d,\.]+(?:\s*(?:milhões?|mil))?)', texto)
    percentuais = re.findall(r'(\d+)%', texto)
    numeros = re.findall(r'\b(\d{1,3}(?:[,\.]\d{3})*(?:[,\.]\d+)?)\b', texto)
    
    return {
        'valores_reais': valores_reais,
        'percentuais': [int(p) for p in percentuais],
        'numeros': numeros
    }

# Extraindo dados do nosso documento
dados_numericos = extrair_dados_numericos(texto_completo)
print("💰 Dados Numéricos Encontrados:")
print(f"Valores em Reais: {dados_numericos['valores_reais']}")
print(f"Percentuais: {dados_numericos['percentuais']}")
print(f"Outros números: {dados_numericos['numeros'][:10]}...")  # Primeiros 10

# Criando visualizações
plt.style.use('seaborn-v0_8')
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Dashboard Automático do Documento', fontsize=16, fontweight='bold')

# Gráfico 1: Percentuais encontrados
if dados_numericos['percentuais']:
    labels = ['Crescimento', 'Retenção', 'Market Share']
    valores = dados_numericos['percentuais'][:3]  # Primeiros 3 percentuais
    
    # Garantir que temos dados suficientes
    while len(valores) < 3:
        valores.append(0)
    
    ax1.bar(labels[:len(valores)], valores, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    ax1.set_title('📈 Principais Métricas (%)')
    ax1.set_ylabel('Percentual')
    
    # Adicionando valores no topo das barras
    for i, v in enumerate(valores):
        ax1.text(i, v + 1, f'{v}%', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Distribuição de Sentimentos (simulado)
sentimentos = ['Positivo', 'Neutro', 'Negativo']
scores = [70, 20, 10]  # Baseado na análise do texto
colors = ['#2ECC71', '#F39C12', '#E74C3C']

ax2.pie(scores, labels=sentimentos, colors=colors, autopct='%1.1f%%', startangle=90)
ax2.set_title('🎭 Análise de Sentimento')

# Gráfico 3: Timeline de métricas (exemplo)
meses = ['Q1', 'Q2', 'Q3', 'Q4']
crescimento = [10, 15, 20, 25]  # Dados do documento

ax3.plot(meses, crescimento, marker='o', linewidth=3, markersize=8, color='#9B59B6')
ax3.fill_between(meses, crescimento, alpha=0.3, color='#9B59B6')
ax3.set_title('📈 Tendência de Crescimento')
ax3.set_ylabel('Crescimento (%)')
ax3.grid(True, alpha=0.3)

# Gráfico 4: Word Cloud das palavras mais importantes
wordcloud = WordCloud(
    width=400, height=300,
    background_color='white',
    colormap='viridis',
    max_words=50
).generate(texto_completo)

ax4.imshow(wordcloud, interpolation='bilinear')
ax4.axis('off')
ax4.set_title('☁️ Palavras-chave Principais')

plt.tight_layout()
plt.show()

print("🎨 Visualizações geradas automaticamente!")

## 🤔 Agente 4: Sistema de Perguntas e Respostas

Este é nosso **gênio da lâmpada**! Você faz uma pergunta sobre qualquer coisa do documento e ele responde na lata!

Usando RAG (Retrieval Augmented Generation) que aprendemos no Módulo 10!

In [ ]:
# Criando sistema de Q&A com RAG
qa_template = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um assistente especialista em análise de documentos.
    Use APENAS as informações dos documentos fornecidos para responder.
    
    Se a informação não estiver nos documentos, diga claramente:
    "Esta informação não está disponível nos documentos analisados."
    
    Seja preciso, cite números quando relevante e forneça contexto.
    """),
    ("human", """
    Contexto dos documentos:
    {contexto}
    
    Pergunta: {pergunta}
    
    Resposta baseada apenas no contexto:
    """)
])

def fazer_pergunta(pergunta, k=3):
    """
    Faz uma pergunta sobre os documentos usando RAG
    """
    # Busca documentos relevantes
    docs_relevantes = vectorstore.similarity_search(pergunta, k=k)
    
    # Junta o contexto
    contexto = "\n\n".join([doc.page_content for doc in docs_relevantes])
    
    # Cria a chain
    chain = qa_template | llm
    
    try:
        resposta = chain.invoke({
            "contexto": contexto,
            "pergunta": pergunta
        })
        return resposta.content, docs_relevantes
    except Exception as e:
        return f"Erro: {e}", []

# Testando com algumas perguntas
perguntas_teste = [
    "Qual foi o crescimento de vendas no Q4?",
    "Quantos novos clientes foram adquiridos?",
    "Quais são os principais desafios identificados?",
    "Qual é a receita total mencionada?"
]

print("🤖 SISTEMA DE PERGUNTAS E RESPOSTAS")
print("=" * 60)

for pergunta in perguntas_teste:
    print(f"\n❓ {pergunta}")
    resposta, docs = fazer_pergunta(pergunta)
    print(f"💡 {resposta}")
    print("-" * 40)

## 🎯 Sistema Multi-Agente Integrado

Agora vamos juntar **todos os agentes** num sistema único! É como orquestrar uma banda onde cada músico toca sua parte, mas o resultado final é uma sinfonia!

**Dica!** Aqui usamos os conceitos de Agents e Tools do Módulo 11!

In [ ]:
class SistemaMultiAgente:
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.relatorio_final = {}
    
    def processar_documento_completo(self, texto):
        """
        Executa análise completa com todos os agentes
        """
        print("🚀 Iniciando análise multi-agente...")
        
        # Agente 1: Extração
        print("\n🔍 Agente 1: Extraindo informações...")
        info_extraida = extrair_informacoes(texto)
        
        # Agente 2: Análise de Sentimento
        print("🎭 Agente 2: Analisando sentimentos...")
        analise = analisar_sentimento(texto)
        
        # Agente 3: Visualizações (já criamos acima)
        print("📊 Agente 3: Gerando visualizações...")
        dados_viz = extrair_dados_numericos(texto)
        
        # Compilando relatório final
        self.relatorio_final = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'informacoes_extraidas': info_extraida,
            'analise_sentimento': analise,
            'dados_numericos': dados_viz,
            'sistema_qa': 'Ativo e funcionando'
        }
        
        return self.relatorio_final
    
    def gerar_resumo_executivo(self):
        """
        Gera um resumo executivo baseado em todas as análises
        """
        resumo_template = ChatPromptTemplate.from_messages([
            ("system", """
            Você é um consultor sênior criando um resumo executivo.
            
            Com base em todas as análises realizadas, crie um resumo executivo
            profissional com:
            
            1. SÍNTESE DOS PRINCIPAIS ACHADOS
            2. MÉTRICAS-CHAVE
            3. INSIGHTS ESTRATÉGICOS
            4. RECOMENDAÇÕES
            5. PRÓXIMOS PASSOS
            
            Use linguagem executiva, seja conciso e foque no que importa para tomada de decisão.
            """),
            ("human", "Dados das análises: {dados_completos}")
        ])
        
        chain = resumo_template | self.llm
        
        try:
            resposta = chain.invoke({
                "dados_completos": str(self.relatorio_final)
            })
            return resposta.content
        except Exception as e:
            return f"Erro ao gerar resumo: {e}"

# Inicializando nosso sistema multi-agente
sistema = SistemaMultiAgente(vectorstore, llm)

# Executando análise completa
relatorio = sistema.processar_documento_completo(texto_completo)

print("\n✅ Análise multi-agente concluída!")
print(f"📄 Relatório gerado às: {relatorio['timestamp']}")

## 📋 Relatório Executivo Final

Agora vamos gerar o **gran finale**: um relatório executivo completo que junta tudo que nossos agentes descobriram!

In [ ]:
# Gerando o resumo executivo
resumo_executivo = sistema.gerar_resumo_executivo()

print("📊 RELATÓRIO EXECUTIVO FINAL")
print("=" * 80)
print(resumo_executivo)
print("=" * 80)

# Salvando relatório completo em JSON
relatorio_completo = {
    'documento_analisado': 'Relatório de Vendas Q4 2024',
    'timestamp': datetime.now().isoformat(),
    'resumo_executivo': resumo_executivo,
    'detalhes': relatorio
}

# Salvando em arquivo
with open('relatorio_multiagente.json', 'w', encoding='utf-8') as f:
    json.dump(relatorio_completo, f, ensure_ascii=False, indent=2)

print("\n💾 Relatório salvo em 'relatorio_multiagente.json'")
print("\n🎉 Sistema multi-agente funcionando perfeitamente!")

## 🎮 Exercício Prático 1

**Desafio**: Crie seu próprio documento de teste!

1. Escreva um texto sobre qualquer assunto (empresa, projeto, análise)
2. Inclua dados numéricos (percentuais, valores, datas)
3. Execute todo o pipeline multi-agente
4. Compare os resultados!

**Dica!** Pode ser sobre sua empresa, um projeto fictício, ou até análise de um filme!

In [ ]:
# SEU CÓDIGO AQUI!
# Crie um novo documento e teste o sistema

meu_documento = """
# Substitua este texto pelo seu documento personalizado
# Lembre-se de incluir:
# - Dados numéricos
# - Datas
# - Informações que possam ser analisadas
"""

# Descomente as linhas abaixo quando tiver seu documento pronto:
# novo_processor = DocumentProcessor()
# novos_chunks = novo_processor.process_text(meu_documento, "meu_teste")
# # Continue o processamento...

print("✏️ Escreva seu documento e execute a análise!")

## 🔧 Melhorias e Extensões

Nosso sistema já está **funcionando**, mas sempre dá pra melhorar! Aqui estão algumas ideias pra turbinar ainda mais:

### 🚀 Próximos Níveis:

1. **Análise de múltiplos documentos** simultaneamente
2. **Comparação temporal** (documentos de períodos diferentes)
3. **Alertas automáticos** quando métricas passam de limites
4. **Integração com APIs** externas (bancos de dados, CRM)
5. **Interface web** com Streamlit (que vamos ver no próximo módulo!)

**Dica!** No Módulo 14 vamos transformar isso numa aplicação web linda!

In [ ]:
# Função para comparar múltiplos documentos
def comparar_documentos(doc1, doc2, titulo1="Documento 1", titulo2="Documento 2"):
    """
    Compara dois documentos lado a lado
    """
    print(f"🔄 Comparando {titulo1} vs {titulo2}")
    
    # Análise do primeiro documento
    info1 = extrair_informacoes(doc1)
    sent1 = analisar_sentimento(doc1)
    
    # Análise do segundo documento
    info2 = extrair_informacoes(doc2)
    sent2 = analisar_sentimento(doc2)
    
    # Template para comparação
    comparacao_template = ChatPromptTemplate.from_messages([
        ("system", """
        Você é um analista comparativo especialista.
        
        Compare os dois documentos e identifique:
        1. PRINCIPAIS DIFERENÇAS
        2. TENDÊNCIAS OPOSTAS
        3. PONTOS EM COMUM
        4. INSIGHTS COMPARATIVOS
        5. RECOMENDAÇÕES BASEADAS NA COMPARAÇÃO
        
        Seja específico e use dados concretos das análises.
        """),
        ("human", """
        Análise Documento 1 ({titulo1}):
        {analise1}
        
        Análise Documento 2 ({titulo2}):
        {analise2}
        
        Faça uma comparação detalhada:
        """)
    ])
    
    chain = comparacao_template | llm
    
    try:
        resultado = chain.invoke({
            "titulo1": titulo1,
            "titulo2": titulo2,
            "analise1": f"{info1}\n\n{sent1}",
            "analise2": f"{info2}\n\n{sent2}"
        })
        return resultado.content
    except Exception as e:
        return f"Erro na comparação: {e}"

# Exemplo de uso (quando você tiver dois documentos)
print("🔍 Função de comparação criada!")
print("📝 Use comparar_documentos(doc1, doc2) para comparar textos")

## 📊 Métricas de Performance do Sistema

Vamos medir quão **eficiente** nosso sistema está sendo! É importante sempre monitorar performance em sistemas de produção.

In [ ]:
import time
from collections import Counter

class MetricasPerformance:
    def __init__(self):
        self.tempos_execucao = []
        self.tokens_processados = 0
        self.consultas_realizadas = 0
        self.inicio_sessao = time.time()
    
    def medir_tempo_execucao(self, funcao, *args, **kwargs):
        """Mede tempo de execução de uma função"""
        inicio = time.time()
        resultado = funcao(*args, **kwargs)
        fim = time.time()
        
        tempo_execucao = fim - inicio
        self.tempos_execucao.append(tempo_execucao)
        
        return resultado, tempo_execucao
    
    def estatisticas_performance(self):
        """Retorna estatísticas de performance"""
        if not self.tempos_execucao:
            return "Nenhuma medição realizada ainda"
        
        tempo_total_sessao = time.time() - self.inicio_sessao
        tempo_medio = np.mean(self.tempos_execucao)
        tempo_total_processamento = sum(self.tempos_execucao)
        
        stats = {
            'tempo_sessao_total': f"{tempo_total_sessao:.2f}s",
            'tempo_processamento_total': f"{tempo_total_processamento:.2f}s",
            'tempo_medio_por_operacao': f"{tempo_medio:.2f}s",
            'operacoes_realizadas': len(self.tempos_execucao),
            'operacoes_por_minuto': f"{len(self.tempos_execucao) / (tempo_total_sessao / 60):.1f}"
        }
        
        return stats

# Inicializando métricas
metricas = MetricasPerformance()

# Testando performance com algumas operações
print("⚡ Testando Performance do Sistema")
print("=" * 50)

# Teste 1: Extração de informações
resultado, tempo = metricas.medir_tempo_execucao(extrair_informacoes, texto_completo)
print(f"🔍 Extração de informações: {tempo:.2f}s")

# Teste 2: Análise de sentimento
resultado, tempo = metricas.medir_tempo_execucao(analisar_sentimento, texto_completo)
print(f"🎭 Análise de sentimento: {tempo:.2f}s")

# Teste 3: Pergunta e resposta
resultado, tempo = metricas.medir_tempo_execucao(fazer_pergunta, "Qual foi o crescimento?")
print(f"❓ Pergunta e resposta: {tempo:.2f}s")

# Estatísticas finais
stats = metricas.estatisticas_performance()
print("\n📊 ESTATÍSTICAS DE PERFORMANCE:")
for chave, valor in stats.items():
    print(f"{chave}: {valor}")

## 🎮 Exercício Final: Sistema Personalizado

**Mega Desafio**: Crie um sistema multi-agente para **seu domínio específico**!

Escolha uma das opções:

1. **Sistema de RH**: Analisa currículos e perfis de candidatos
2. **Sistema Financeiro**: Analisa relatórios de investimento
3. **Sistema de Marketing**: Analisa campanhas e resultados
4. **Sistema Acadêmico**: Analisa papers e pesquisas

**Requisitos**:
- Pelo menos 3 agentes especializados
- Sistema de perguntas customizado
- Visualizações específicas do domínio
- Relatório final personalizado

In [ ]:
# SEU SISTEMA PERSONALIZADO AQUI!

# Exemplo de estrutura:
class MeuSistemaEspecializado:
    def __init__(self):
        # Seus agentes especializados
        pass
    
    def agente_especializado_1(self, texto):
        # Seu primeiro agente especializado
        pass
    
    def agente_especializado_2(self, texto):
        # Seu segundo agente especializado  
        pass
    
    def gerar_relatorio_personalizado(self):
        # Seu relatório personalizado
        pass

print("🚀 Crie seu sistema personalizado aqui!")
print("💡 Use como base tudo que aprendemos, mas adapte pro seu domínio!")

## 🎊 Resumão do Projeto Final 2

**Liiindo!** Você acabou de criar um sistema multi-agente completo! 🎉

### 🏆 O que conquistamos:

✅ **Sistema Multi-Agente** com 4 especialistas
✅ **Extração automática** de informações
✅ **Análise de sentimento** avançada
✅ **Visualizações automáticas** dos dados
✅ **Sistema de Q&A** com RAG
✅ **Relatórios executivos** automáticos
✅ **Métricas de performance**

### 🔗 Conceitos do LangChain utilizados:

- **ChatModels** (Módulo 2) ✅
- **LCEL** (Módulo 3) ✅  
- **Prompt Templates** (Módulo 4) ✅
- **Chains** (Módulo 6) ✅
- **Memory Systems** (Módulo 7) ✅
- **Document Loading** (Módulo 8) ✅
- **Vector Stores** (Módulo 9) ✅
- **RAG** (Módulo 10) ✅
- **Agents** (Módulo 11) ✅

### 🚀 Próximos passos:

**Módulo 14**: Vamos transformar este sistema numa **aplicação web linda** com Streamlit!

**Dica!** Guarde bem este código, pois vamos usar como base pro deploy!

## 🎯 Exercício Bônus: Otimização de Custos

Em produção, é importante **otimizar custos** de API. Vamos criar um sistema que monitora e otimiza o uso de tokens!

In [ ]:
class OtimizadorCustos:
    def __init__(self):
        self.tokens_utilizados = 0
        self.custo_por_token = 0.000002  # Exemplo de custo
        self.cache_respostas = {}  # Cache simples
    
    def calcular_tokens_aproximados(self, texto):
        """Estimativa rápida de tokens (1 token ≈ 4 caracteres)"""
        return len(texto) // 4
    
    def verificar_cache(self, pergunta):
        """Verifica se já temos resposta em cache"""
        return self.cache_respostas.get(pergunta)
    
    def adicionar_cache(self, pergunta, resposta):
        """Adiciona resposta ao cache"""
        self.cache_respostas[pergunta] = resposta
    
    def fazer_pergunta_otimizada(self, pergunta):
        """Pergunta otimizada com cache"""
        # Verifica cache primeiro
        resposta_cache = self.verificar_cache(pergunta)
        if resposta_cache:
            print(f"💾 Resposta encontrada em cache! Economia de tokens.")
            return resposta_cache
        
        # Se não tem cache, faz a pergunta
        resposta, docs = fazer_pergunta(pergunta)
        tokens_usados = self.calcular_tokens_aproximados(pergunta + resposta)
        
        # Atualiza métricas
        self.tokens_utilizados += tokens_usados
        
        # Salva no cache
        self.adicionar_cache(pergunta, resposta)
        
        print(f"🏷️ Tokens utilizados: {tokens_usados}")
        print(f"💰 Custo estimado: ${tokens_usados * self.custo_por_token:.6f}")
        
        return resposta
    
    def relatorio_custos(self):
        """Relatório de custos acumulados"""
        custo_total = self.tokens_utilizados * self.custo_por_token
        return {
            'tokens_total': self.tokens_utilizados,
            'custo_total': f"${custo_total:.6f}",
            'respostas_em_cache': len(self.cache_respostas),
            'economia_potencial': f"${len(self.cache_respostas) * 100 * self.custo_por_token:.6f}"
        }

# Testando otimização
otimizador = OtimizadorCustos()

print("💰 TESTE DE OTIMIZAÇÃO DE CUSTOS")
print("=" * 50)

# Primeira pergunta
resp1 = otimizador.fazer_pergunta_otimizada("Qual foi o crescimento?")

# Segunda pergunta (mesma - deve usar cache)
resp2 = otimizador.fazer_pergunta_otimizada("Qual foi o crescimento?")

# Relatório final
relatorio = otimizador.relatorio_custos()
print("\n📊 RELATÓRIO DE CUSTOS:")
for chave, valor in relatorio.items():
    print(f"{chave}: {valor}")

print("\n💡 Sistema de cache funcionando! Economia garantida em produção!")